In [ ]:
"""
AWS Filtering Script for Operation IceBridge Airborne Topographic Mapper L2 Icessn Elevation, Slope, and Roughness, v.2 Data

This script filters ILATM2 data to within 5km of on-ice PROMIC and GC-NET automated weather stations. 
Shapefiles for the AWS locations and Greenland coastline was sourced from QGreenland. 
"""

In [ ]:
# Imports necessary libraries

import pandas as pd
import geopandas as gpd
import os

In [ ]:
coastline_gpd = gpd.read_file("./greenlandCoastline.shp") # Load coastline .shp 
GCNET_gdf = gpd.read_file("./WxStations/GCNETSites.shp") # Load GCNET .shps 
PROMICE_gdf = gpd.read_file("./WxStations/PROMICESites.shp") # Load PROMICE .shps 

GCNET_gdf = GCNET_gdf.to_crs(coastline_gpd.crs) # Set weather stations to coastline CRS 
PROMICE_gdf = PROMICE_gdf.to_crs(coastline_gpd.crs)

GCNET_gdf["buffered_5km"] = GCNET_gdf.buffer(5000) # Buffer station locations with a 5 km radius 
PROMICE_gdf["buffered_5km"] = PROMICE_gdf.buffer(5000)

weather_stations = gpd.GeoDataFrame(pd.concat([GCNET_gdf, PROMICE_gdf], ignore_index=True), crs=coastline_gpd.crs) # Concatenate PROMICE and GC-NET stations 
 
main_folder_path = "./weather_stations/"   
output_folder_path = "./weather_stations_5km/"

os.makedirs(output_folder_path, exist_ok=True) # Create output directory if it doesn't exist

In [ ]:
for csv_file in os.listdir(main_folder_path): # Process each buffered .csv
    if csv_file.endswith('.csv'): # Check if the file has a '.csv' extension
        csv_file_path = os.path.join(main_folder_path, csv_file) # Create the full file path
        print(f"Processing file: {csv_file_path}")

        df = pd.read_csv(csv_file_path) # Read .csvs into a DataFrame 
        df = df.apply(pd.to_numeric, errors='coerce').dropna() # Convert columns to numeric
        df = df.rename(str.strip, axis='columns')

        tmp_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitude(deg)'], df['Latitude(deg)']), crs='EPSG:4326') # Convert to DFs to GDFs 
        tmp_gdf = tmp_gdf.to_crs(coastline_gpd.crs) # Reproject GDF to coastline CRS 

        clipped_data = [] # Initialize list to store clipped data 
        for i in range(len(weather_stations)): # Loop through each station and clip the data within the 5km buffer of the station
            clipped_gdf = gpd.clip(tmp_gdf, weather_stations.loc[i, "buffered_5km"]) 
            
            if not clipped_gdf.empty: # If the clipped data is not empty, append it to the list
                clipped_data.append(clipped_gdf)

        if clipped_data: # Cncatenate the DataFrames and save the result
            out = pd.concat(clipped_data).drop(columns='geometry') 
            output_file_path = os.path.join(output_folder_path, csv_file.replace('.csv', '_5km_buffered.csv')) # Create output file path 
            out.to_csv(output_file_path, index=False) # Save to a .csv 
            print(f"File saved: {output_file_path}")

        else: 
            print(f"No data after clipped for file: {csv_file}") # If no data was saved, print a message

